In [1]:
## CELL 1: LOAD ###
# run this cell to load your datafile through the GUI (currently only works with .acq files, any template)
# select:
# (1) The channel containing acceleration
# (2) The channel containing respiration 
# (3) If you need respiration estiamted (e.g., from the z0 timeseries) using a low-pass filter

!pip install bioread
from miniMEAPsys import loadem
cont_dict,file_path=loadem()
print('data loaded!')

selected contractility channel is dZ
selected respiration channel is Respiration
extracting raw signals from AcqKnowledge files...
removing slow movements from acceleration data with 7-order polynomial
applying lowpass filter to acceleration channel, 22.5 Hz cutoff
removing slow movements from respiration data with 7-order polynomial
applying lowpass filter to respiration channel, 0.35 Hz cutoff
estimating respiration amount and cycle...
data loaded!


In [4]:
## CELL 2: SET THRESHOLD AND ESTIMATE PEAKS ###
# run this cell to open a plot to decide minimum peak height in the acceleration timeseries
# Enter value (will be around ~0.5) in the accompanying GUI.
# programme will then do its first estimate on peak values
import matplotlib.pyplot as plt
from miniMEAPsys import compute_peaks,thresh_ind

%matplotlib tk
plt.close('all')
i=thresh_ind(cont_dict['hz'])
plt.plot(cont_dict['t'][i],cont_dict['s'][i])
peak_times,peak_vals,peak_inds=compute_peaks(cont_dict)

plt.close('all')

In [5]:
## CELL 3: INTERACTIVE PLOT - PEAKS IN ACCELERATION TIME SERIES###
# run this cell to open the interactive plot to visualise and amend peaks in the acceleration time series
# accleration peaks are more robust to noise and are better estimates of heartbeat times
# left click - add peak
# right click - remove peak
# hold m and left click - use moving average to estimate peak
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from miniMEAPsys import *

%matplotlib tk
plt.close('all')
fig,ax = plt.subplots()
plt.subplots_adjust(bottom=0.25)

w = ax.plot(cont_dict['t'],cont_dict['s'])
peak_plot, = ax.plot(peak_times,peak_vals ,'vm')

# Set the axis and slider position in the plot
bottom_pos = plt.axes([0.2, 0.1, 0.65, 0.03],facecolor = 'white')
scroll_slider = Slider(bottom_pos,'time', -1,cont_dict['t'][-1])

# Make a vertically oriented slider to control the amplitude
right_pos = plt.axes([.95, 0.25, 0.0225, 0.63])
y_zoom = Slider(right_pos,label="Y zoom",valmin=0,valmax=1,valinit=.5,orientation="vertical")
left_pos = plt.axes([.05, 0.25, 0.0225, 0.63])
x_zoom = Slider(left_pos,label="X zoom",valmin=0,valmax=1,valinit=.5,orientation="vertical")

ax.set_ylim(-np.median(peak_vals)*4*.5,np.median(peak_vals)*4*.5)

ax.set_xlim(-1,9)

orig_n = len(peak_times)

ax.set_title('Cell 3: remove / add acceleration peaks. Orig: '+str(orig_n)+', Updated: '+str(orig_n))

def update(val):
    pos = scroll_slider.val
    yzoom = y_zoom.val
    xzoom = x_zoom.val
    ax.set_xlim(pos, pos+20*xzoom)
    fig.canvas.draw_idle
    ecc=np.median(peak_vals)*4*yzoom
    ax.set_ylim(-ecc, ecc)
    fig.canvas.draw_idle
    
# update function called using on_changed() function
scroll_slider.on_changed(update)
y_zoom.on_changed(update)
x_zoom.on_changed(update)

# Display the plot
plt.show()

def onclick(event):
    global peak_times,peak_vals,fig,peak_plot,orig_n,ax
    ix, iy, ib, ik, iax = event.xdata, event.ydata, event.button, event.key, event.inaxes
    if (ix!=None) & (iax==ax):
        if (np.min(np.abs(ix-peak_times))<100) & (event.button!=1) & (ik==None):
            peak_times,peak_vals,peak_plot=remove_point(ix,peak_times,peak_vals,peak_plot)
            ax.set_title('Cell 3: remove / add acceleration peaks. Orig: '+str(orig_n)+', Updated: '+str(len(peak_vals)))
        elif (event.button==1) & (ik==None) & (ix>0) & (ix>0) & (iy<np.max(peak_vals)*1.2): #update dist from pos
            peak_times,peak_vals,peak_plot=add_point(ix,iy,peak_times,peak_vals,peak_plot)
            ax.set_title('Cell 3: remove / add acceleration peaks. Orig: '+str(orig_n)+', Updated: '+str(len(peak_vals)))
        elif (event.button==1)&(ik=='m'):
            peak_times,peak_vals,peak_plot=meap_dat(ix,iy,peak_times,peak_vals,peak_plot)
            ax.set_title('Cell 3: remove / add acceleration peaks. Orig: '+str(orig_n)+', Updated: '+str(len(peak_vals)))
        fig.canvas.draw()
        fig.canvas.flush_events()
        
        if not plt.fignum_exists(1):
            canvas.mpl_disconnect(cid)
            canvas.mpl_disconnect(cidk)
    return peak_times,peak_vals

def onarrow(e):
    global scroll_slider
    if e.key == "right":
        pos = ax.get_xlim()
        scroll_slider.set_val(pos[0]+.500)
    elif e.key == "left":
        pos = ax.get_xlim()
        scroll_slider.set_val(pos[0]-.500)
    else:
        return

cid = fig.canvas.mpl_connect('button_press_event', onclick)
cidk = fig.canvas.mpl_connect('key_press_event', onarrow)

In [6]:
## CELL 4: INTERACTIVE PLOT - FINAL CHECK - PEAKS IN CONTRACTILITY TIME SERIES###
# run this cell to open the interactive plot to visualise and amend peak amplitudes in the contractility time series
# you can no longer remove and add peaks to the dataset, only adjust their amplitudes
# left click - adjust amplitude
# hold m and left click - use moving average to estimate peak amplitude
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from miniMEAPsys import *

##Store acceleration peaks and times
cont_dict['acc_peaks']=peak_vals
cont_dict['acc_peak_times']=peak_times

%matplotlib tk
plt.close('all')
fig,ax = plt.subplots()
plt.subplots_adjust(bottom=0.25)

#estimating heartrate before estimating contractility
cont_dict['RR']=np.hstack([np.nan,np.diff(peak_times)])

#estimate contractility from acceleration and adjust times etc.
cont_s = np.diff(cont_dict['s'])
cont_t = cont_dict['t'][1:]
init_cont_inds = peak_inds+1
cont_inds = shift_peaks_no_plot(250,peak_times,cont_t,cont_s,init_cont_inds)
peak_times = cont_t[cont_inds]
peak_vals = cont_s[cont_inds]

w = ax.plot(cont_t,cont_s)
peak_plot, = ax.plot(peak_times,peak_vals ,'vm')

# Set the axis and slider position in the plot
bottom_pos = plt.axes([0.2, 0.1, 0.65, 0.03],facecolor = 'white')
scroll_slider = Slider(bottom_pos,'time', -1,cont_t[-1])

# Make a vertically oriented slider to control the amplitude
right_pos = plt.axes([.95, 0.25, 0.0225, 0.63])
y_zoom = Slider(right_pos,label="Y zoom",valmin=0,valmax=.1,valinit=.05,orientation="vertical")
left_pos = plt.axes([.05, 0.25, 0.0225, 0.63])
x_zoom = Slider(left_pos,label="X zoom",valmin=0,valmax=1,valinit=.5,orientation="vertical")

ax.set_ylim(-(np.median(peak_vals)+.05),np.median(peak_vals)+.05)

ax.set_xlim(-1,9)

ax.set_title('Cell 4: adjust contractility amplitudes only')

def update(val):
    pos = scroll_slider.val
    yzoom = y_zoom.val
    xzoom = x_zoom.val
    ax.set_xlim(pos, pos+20*xzoom)
    fig.canvas.draw_idle
    ecc=np.median(peak_vals)+yzoom
    ax.set_ylim(-ecc, ecc)
    fig.canvas.draw_idle
    
# update function called using on_changed() function
scroll_slider.on_changed(update)
y_zoom.on_changed(update)
x_zoom.on_changed(update)

# Display the plot
plt.show()

def onclick(event):
    global peak_times,peak_vals,fig,peak_plot
    ix, iy, ib, ik, iax = event.xdata, event.ydata, event.button, event.key, event.inaxes
    if (ix!=None) & (iax==ax):
        if (event.button==1) & (ik==None) & (ix>0) & (ix>0) & (iy<np.max(peak_vals)*1.2): 
            peak_times,peak_vals,peak_plot=adjust_peak_amp(ix,iy,peak_times,peak_vals,peak_plot)
        elif (event.button==1)&(ik=='m'):
            peak_times,peak_vals,peak_plot=meap_dat_cell4(ix,iy,peak_times,peak_vals,peak_plot)
        fig.canvas.draw()
        fig.canvas.flush_events()
        
        if not plt.fignum_exists(1):
            canvas.mpl_disconnect(cid)
            canvas.mpl_disconnect(cidk)
            
    return peak_times,peak_vals

def onarrow(e):
    global scroll_slider
    if e.key == "right":
        pos = ax.get_xlim()
        scroll_slider.set_val(pos[0]+.500)
    elif e.key == "left":
        pos = ax.get_xlim()
        scroll_slider.set_val(pos[0]-.500)
    else:
        return

cid = fig.canvas.mpl_connect('button_press_event', onclick)
cidk = fig.canvas.mpl_connect('key_press_event', onarrow)

In [7]:
## CELL 5: OUTPUT DATA TO CSV ###
# this cell will model respiration and heart-rate out of the beatwise contractility estimates and output to a csv
import pandas as pd
from miniMEAPsys import regress_out,outsave_box
plt.close('all')
cont_dict['cont_peak_times']=peak_times
cont_dict['raw_contractility_peaks']=peak_vals
cont_dict=regress_out(cont_dict)

out_f = outsave_box(file_path[:-4]+'.csv')

out_dict={}
out_dict['time']=cont_dict['cont_peak_times']
out_dict['contractility']=cont_dict['resid_contractility']
pd.DataFrame(out_dict).to_csv(out_f)

',time,contractility\n0,0.48300010223259665,0.05331758195901956\n1,1.389000221591788,0.06079258221742841\n2,2.316000343717583,0.070149693872203\n3,3.2810004708496137,0.06391243066148782\n4,4.262000600089532,0.059440125570812644\n5,5.244000729461195,0.06563210948058272\n6,6.221000858174142,0.05499507282012607\n7,7.178000984252228,0.06233421697961549\n8,8.089001104270135,0.05182031359679291\n9,8.96300121941355,0.06406522162396086\n10,9.864001338114026,0.06024057832946578\n11,10.828001465114314,0.07761648122880159\n12,11.892001605288906,0.06681377725620036\n13,12.87500173479231,0.0689253519732799\n14,14.129001899998078,0.06131315274170214\n15,15.07100202410002,0.09122966882006547\n16,15.445002073371915,0.08686612189390874\n17,17.211002306030117,0.05728331411731755\n18,18.212002437904896,0.07037970841464003\n19,19.156002562270324,0.060503918524850236\n20,20.074002683210434,0.04964586847308033\n21,20.98800280362357,0.0519200566873667\n22,21.920002926408078,0.06658647766799104\n23,22.8310030